In [17]:
from memorizz.database.mongodb import mongodb_tools, MongoDBToolsConfig, MongoDBTools, get_mongodb_toolbox

In [25]:
import os
import getpass

OPENAI_API_KEY = getpass.getpass("OpenAI API Key: ")
os.environ["OPENAI_API_KEY"] = OPENAI_API_KEY

MONGO_URI = getpass.getpass("Enter MongoDB URI: ")
os.environ["MONGO_URI"] = MONGO_URI

GPT_MODEL = "gpt-4o"

In [26]:
# Option 1: Create MongoDBTools instance and get the decorator
config = MongoDBToolsConfig(mongo_uri=MONGO_URI)
mongodb_tools = MongoDBTools(config)
# mongodb_toolbox = mongodb_tools.mongodb_toolbox

In [27]:
# Option 2: Use the get_mongodb_toolbox function
mongodb_toolbox = get_mongodb_toolbox(config)

In [40]:
import random
from datetime import datetime

@mongodb_toolbox()
def shout(statement: str) -> str:
  """
  Convert a statement to uppercase letters to emulate shouting. Use this when a user wants to emphasize something strongly or when they explicitly ask to 'shout' something..

  """
  return statement.upper()

@mongodb_toolbox()
def get_weather(location: str, unit: str = "celsius") -> str:
    """
    Get the current weather for a specified location.
    Use this when a user asks about the weather in a specific place.

    :param location: The name of the city or location to get weather for.
    :param unit: The temperature unit, either 'celsius' or 'fahrenheit'. Defaults to 'celsius'.
    :return: A string describing the current weather.
    """
    conditions = ["sunny", "cloudy", "rainy", "snowy"]
    temperature = random.randint(-10, 35)

    if unit.lower() == "fahrenheit":
        temperature = (temperature * 9/5) + 32

    condition = random.choice(conditions)
    return f"The weather in {location} is currently {condition} with a temperature of {temperature}°{'C' if unit.lower() == 'celsius' else 'F'}."

@mongodb_toolbox()
def get_stock_price(symbol: str) -> str:
    """
    Get the current stock price for a given stock symbol.
    Use this when a user asks about the current price of a specific stock.

    :param symbol: The stock symbol to look up (e.g., 'AAPL' for Apple Inc.).
    :return: A string with the current stock price.
    """
    price = round(random.uniform(10, 1000), 2)
    return f"The current stock price of {symbol} is ${price}."

@mongodb_toolbox()
def get_current_time(timezone: str = "UTC") -> str:
    """
    Get the current time for a specified timezone.
    Use this when a user asks about the current time in a specific timezone.

    :param timezone: The timezone to get the current time for. Defaults to 'UTC'.
    :return: A string with the current time in the specified timezone.
    """
    current_time = datetime.utcnow().strftime("%H:%M:%S")
    return f"The current time in {timezone} is {current_time}."


INFO:httpx:HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 400 Bad Request"
ERROR:memorizz.database.mongodb.mongodb_tools:Error generating embedding: Error code: 400 - {'error': {'message': "'$.input' is invalid. Please check the API reference: https://platform.openai.com/docs/api-reference.", 'type': 'invalid_request_error', 'param': None, 'code': None}}
ERROR:memorizz.database.mongodb.mongodb_tools:Error registering tool shout: Error code: 400 - {'error': {'message': "'$.input' is invalid. Please check the API reference: https://platform.openai.com/docs/api-reference.", 'type': 'invalid_request_error', 'param': None, 'code': None}}


BadRequestError: Error code: 400 - {'error': {'message': "'$.input' is invalid. Please check the API reference: https://platform.openai.com/docs/api-reference.", 'type': 'invalid_request_error', 'param': None, 'code': None}}

In [29]:
def populate_tools(search_results):
  """
  Populate the tools array based on the results from the vector search.

  Args:
  search_results (list): The list of documents returned from the vector search.

  Returns:
  list: A list of tool definitions in the format required by the OpenAI API.
  """
  tools = []
  for result in search_results:
    tool = {
      "type": "function",
      "function": {
        "name": result["name"],
        "description": result["description"],
        "parameters": result["parameters"]
      }
    }
    tools.append(tool)
  return tools

In [31]:
user_query = "Hi, can you shout the statement: We are there"

In [37]:
tools = mongodb_tools.populate_tools(user_query)

INFO:httpx:HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"
INFO:memorizz.database.mongodb.mongodb_tools:Successfully populated 2 tools


In [38]:
import pprint

pprint.pprint(tools)

[{'function': {'description': 'Convert a statement to uppercase letters to '
                              'emulate shouting. Use this when a user wants to '
                              'emphasize something strongly or when they '
                              "explicitly ask to 'shout' something..",
               'name': 'shout',
               'parameters': {'additionalProperties': False,
                              'properties': {'statement': {'description': 'Parameter '
                                                                          'statement',
                                                           'type': 'string'}},
                              'required': ['statement'],
                              'type': 'object'}},
  'type': 'function'},
 {'function': {'description': 'Get the current stock price for a given stock '
                              'symbol.\n'
                              'Use this when a user asks about the current '
                      